## Etapa 3- Análise de dados


In [15]:
from pathlib import Path
import pandas as pd
import sqlite3

CWD = Path.cwd().resolve()
PROJECT_DIR = CWD.parent if CWD.name.lower() in {"scr", "src", "notebooks"} else CWD

DATA_DIR = PROJECT_DIR / "data"
PROC_DIR = DATA_DIR / "processed"
SQLITE_PATH = DATA_DIR / "netflix.db"

conn = sqlite3.connect(SQLITE_PATH.as_posix())

1) Top países com mais títulos
O que a query faz: conta quantos títulos existem por país na tabela normalizada titles_by_country.
Por que nessa tabela? porque eu já “explodi” country (N:N), ou seja, cada país está em uma linha por título.
 Assim, o COUNT(*) reflete de forma correta a presença do título em cada país.
 - Ordeno por total_titles DESC para facilitar a visualização do ranking no gráfico.

In [16]:
top_countries = """
SELECT country, COUNT(*) AS total_titles
FROM titles_by_country
GROUP BY country
ORDER BY total_titles DESC;
"""
df_top_countries = pd.read_sql_query(top_countries, conn)
path1 = PROC_DIR / "top_countries.csv"
df_top_countries.to_csv(path1, index=False, encoding="utf-8")
path1

WindowsPath('C:/Users/maria/OneDrive/Documentos/Portifolio-dados/netflix-analytics/data/processed/top_countries.csv')

2) Evolução por mês-ano  
O que a query faz: agrega os títulos por mês-ano com base em date_added da titles_clean.  
Por que usar strftime('%Y-%m', date_added)? transforma a data (YYYY-MM-DD) em uma etiqueta mensal (YYYY-MM) para agrupar.  
Por que WHERE date_added IS NOT NULL? evita linhas sem data que atrapalhariam a contagem.  
Por que ordenar por year_month? para já sair do banco em ordem cronológica.

In [17]:
qnt_month = """
SELECT
  strftime('%Y-%m', date_added) AS year_month,
  COUNT(*) AS total_titles
FROM titles_clean
WHERE date_added IS NOT NULL
GROUP BY year_month
ORDER BY year_month;
"""
df_trend = pd.read_sql_query(qnt_month, conn)
path2 = PROC_DIR / "trend_year_month.csv"
df_trend.to_csv(path2, index=False, encoding="utf-8")
path2

WindowsPath('C:/Users/maria/OneDrive/Documentos/Portifolio-dados/netflix-analytics/data/processed/trend_year_month.csv')

3) Distribuição filmes x séries
- O que a query faz: conta quantos registros existem por type na base limpa (titles_clean).
- Ordeno por total DESC para facilitar a leitura (maior primeiro).

In [18]:
type_dist = """
SELECT type, COUNT(*) AS total
FROM titles_clean
GROUP BY type
ORDER BY total DESC;
"""
df_type = pd.read_sql_query(type_dist, conn)
path3 = PROC_DIR / "type_distribution.csv"
df_type.to_csv(path3, index=False, encoding="utf-8")
path3


WindowsPath('C:/Users/maria/OneDrive/Documentos/Portifolio-dados/netflix-analytics/data/processed/type_distribution.csv')

4) Elenco — atores/atrizes mais frequentes
Aqui a coluna "cast" é multivalor e vem como uma única string com nomes separados por vírgula
 Para contar aparições individuais, eu preciso separar esses nomes.
Metódo usado: um CTE recursivo (WITH RECURSIVE) em SQLite que vai fatiando a string a cada vírgula:
 - Começo garantindo que sempre exista uma vírgula no final (rest = cast || ',') para simplificar o corte.
 - Em cada passo, pego o token até a próxima vírgula como person e avanço o restante em rest.
 - Repito até rest ficar vazio.

Por que não usei JSON aqui? porque no meu ambiente SQLite obtive "malformed JSON" (strings com caracteres
 problemáticos). O método recursivo por vírgula funciona melhor nesse cenário.

Resultado final: cada ocorrência de pessoa vira uma linha, então o COUNT(*) vira o número de aparições no elenco dos títulos.

In [19]:
cast_counts = """
WITH RECURSIVE split AS (
    -- ponto de partida: pega cada linha e garante uma vírgula no final para facilitar o loop
    SELECT
        show_id,
        TRIM("cast") || ',' AS rest,
        '' AS person
    FROM titles_clean
    WHERE "cast" IS NOT NULL AND TRIM("cast") <> ''

    UNION ALL

    -- em cada passo, recorta o token até a próxima vírgula e avança o resto
    SELECT
        show_id,
        CASE
            WHEN INSTR(rest, ',') > 0 THEN SUBSTR(rest, INSTR(rest, ',') + 1)
            ELSE ''
        END AS rest,
        TRIM(SUBSTR(rest, 1, INSTR(rest, ',') - 1)) AS person
    FROM split
    WHERE rest <> '' AND INSTR(rest, ',') > 0
)
SELECT person, COUNT(*) AS appearances
FROM split
WHERE person <> ''      -- ignora vazios
GROUP BY person
ORDER BY appearances DESC;
"""

df_cast = pd.read_sql_query(cast_counts, conn)
path4 = PROC_DIR / "cast_counts.csv"
df_cast.to_csv(path4, index=False, encoding="utf-8")
print("Exportado:", path4)


Exportado: C:\Users\maria\OneDrive\Documentos\Portifolio-dados\netflix-analytics\data\processed\cast_counts.csv


In [20]:
conn.close()
